In [109]:
!pip install Counter

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
import json

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import altair as alt
import seaborn as sns

import regex as re

import gensim
from gensim.models import Word2Vec
from gensim import corpora, models
# from transformers import pipeline
import string
import nltk
# nltk.download('stopwords')
# import liwc
import collections

import sklearn
from sklearn.manifold import TSNE

# from transformers import BertModel
# from transformers import AutoTokenizer
# from transformers import pipeline

import time
import logging
import warnings
warnings.simplefilter('ignore')

save_progress = "save_progress/"
data = "data/"

C:\Users\grace\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import importlib
import utils
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\grace\\Desktop\\old_classes\\macs_404_patterns\\folktales\\utils.py'>

# Processing story data

In [235]:
stories = pd.read_csv(data+'folk_tales_deduplicated.csv')
stories = stories[['nation','title','text']]
stories = stories.dropna(subset = ['nation'])

### Getting lemmas/phrases

In [236]:
wordlist = [utils.to_wordlist([stories.text[x]]) for x in stories.index]

In [237]:
wordlist_bystory = []
for story in wordlist:
    wordlist_bystory.append([word for chunk in story for word in chunk])
stories['wordlist'] = wordlist_bystory
stories['cleaned'] = [" ".join(stories.wordlist[i]).replace("-"," ") for i in stories.index]
stories['cleaned'] = [re.sub('[^a-zA-Z ]','',x) for x in stories.cleaned]

In [584]:
# stories.loc[stories['cleaned'].str.contains(' thor ')]

In [155]:
# lemmas = stories['cleaned'].apply(utils.get_lemmas)
# lemmas.to_pickle(save_progress+'story_lemmas.pkl')
lemmas = pd.read_pickle(save_progress+'story_lemmas.pkl')
# stories['lemmas'] = lemmas

In [239]:
phrased = utils.make_bigrams(lemmas)
stories['phrased'] = phrased

### Getting language families

In [176]:
with open(data+"grouped_regions.json") as f:
    grouped_regions = json.load(f)

lang_fam = []
for i in stories.index:
    hasfam = False
    for key in grouped_regions.keys():
        if stories.loc[i,'nation'] in grouped_regions[key]:
            lang_fam.append(key)
            hasfam = True
    if not hasfam:
        lang_fam.append(np.nan)
stories['language_family'] = lang_fam

### Saving/reading

In [585]:
stories['sid'] = stories.index
stories['sid'] = stories['sid'].apply(lambda x: 'sid'+str(x))

In [587]:
stories.to_csv(save_progress+'stories.csv',index=False)

In [5]:
stories = pd.read_csv(save_progress+'stories.csv')
sid_nation = {}
for i in stories.index:
    sid_nation[stories['sid'][i]] = stories['nation'][i]
nation_sid = {}
for nation in stories.nation.unique():
    nation_sid[nation] = list(stories.query('nation == @nation')['sid'])
stories.head(2)

,nation,title,text,wordlist,cleaned,lemmas,phrased,language_family,sid
0,japanese,Momotaro,If you’ll believe me there was a time when the...,"['believe', 'time', 'fairies', 'none', 'shy', ...",believe time fairies none shy time beasts talk...,"['believe', 'time', 'fairy', 'none', 'time', '...","['believe', 'time', 'fairy', 'none', 'time', '...",east_asia,sid0
1,serbian,The Birdcatcher,Near Constantinople there lived a man who knew...,"['near', 'constantinople', 'lived', 'man', 'kn...",near constantinople lived man knew occupation ...,"['near', 'constantinople', 'live', 'knew', 'oc...","['near', 'constantinople', 'live', 'knew', 'oc...",slavic,sid1


## Processing Hofstede data

In [616]:
hofstede = pd.read_excel(data+'hofstede.xlsx') # https://www.kaggle.com/datasets/aleksakenjic/hofstedes-cultural-dimensions
language = pd.read_csv(data+'countries_languages.csv') # https://www.kaggle.com/datasets/shubhamptrivedi/languages-spoken-across-various-nations
language['main'] = language['Languages Spoken'].apply(lambda x: x.split(' ')[0].replace(',','').split('/')[0])
hofstede = pd.merge(hofstede,language[['Country','main']],how='left',left_on='country',right_on='Country')

In [33]:
hofstede[hofstede['main'].isna()]

,ctr,country,pdi,idv,mas,uai,ltowvs,ivr,estim,Unnamed: 9,Unnamed: 10,Country,main
11,BOS,Bosnia,90,22,48,87,69.773300,44.196429,1,NaN,NaN,NaN,NaN
21,CZE,Czech Rep,57,58,57,74,70.025189,29.464286,0,NaN,NaN,NaN,NaN
23,DOM,Dominican Rep,65,30,65,45,13.098237,54.241071,1,NaN,NaN,NaN,NaN
34,HOK,Hong Kong,68,25,57,29,60.957179,16.964286,0,NaN,NaN,NaN,NaN
46,KOR,South Korea,60,18,39,85,100.000000,29.464286,0,NaN,NaN,NaN,NaN
62,MAC,North Macedonia,90,22,45,87,62.000000,35.000000,1,NaN,NaN,NaN,NaN
70,PUE,Puerto Rico,68,27,56,38,0.000000,89.955357,1,NaN,NaN,NaN,NaN
73,STP,Sao Tome and Principe,75,37,24,70,32.000000,41.000000,1,NaN,NaN,NaN,NaN
77,SLK,Slovak Rep,100,52,110,51,76.574307,28.348214,0,NaN,NaN,NaN,NaN
89,USA,U.S.A.,40,91,62,46,25.692695,68.080357,0,NaN,NaN,NaN,NaN


In [617]:
hofstede_fillna = {'Bosnia':'bosnian','Czech Rep':'czechoslovak','South Korea':'korean','U.S.A':'English'}
hofstede['main'] = hofstede['main'].apply(lambda x: hofstede_fillna[x] if x in hofstede_fillna.keys() else x)
hofstede = hofstede.groupby('main').mean().reset_index()
hofstede['main'] = hofstede['main'].str.lower()

In [620]:
stories_hofstede = pd.merge(stories[['sid','nation','language_family','title','text']], hofstede[['pdi','idv','mas','uai','ltowvs','ivr','main']], left_on='nation', right_on='main', how='inner').drop(columns='main')
stories_hofstede.to_csv(save_progress+'stories_hofstede.csv',index=False)

In [6]:
stories_hofstede = pd.read_csv(save_progress+'stories_hofstede.csv')
stories_hofstede.head()

,sid,nation,language_family,title,text,pdi,idv,mas,uai,ltowvs,ivr
0,sid0,japanese,east_asia,Momotaro,If you’ll believe me there was a time when the...,54.0,46.0,95.0,92.0,87.90932,41.741071
1,sid65,japanese,east_asia,The Filial Girl,"A girl once lived in the province of Echigo, w...",54.0,46.0,95.0,92.0,87.90932,41.741071
2,sid109,japanese,east_asia,The Tongue-Cut Sparrow,Once upon a time there was an old man who live...,54.0,46.0,95.0,92.0,87.90932,41.741071
3,sid122,japanese,east_asia,The Mallet,There were once two farmer men who were brothe...,54.0,46.0,95.0,92.0,87.90932,41.741071
4,sid210,japanese,east_asia,Karma,"The young man, Ito Tatewaki, was returning hom...",54.0,46.0,95.0,92.0,87.90932,41.741071


# Lemma Frequencies

## 

In [ ]:
noun_counts = utils.make_posdict(lemma_counts,'n')
verb_counts = utils.make_posdict(lemma_counts,'v')

In [169]:
nouns_bynation = utils.group_lemmacounts_bynation(noun_counts,list(stories.nation.unique())).to_csv(save_progress+"nouns_bynation.csv",index=True)
nouns_bynation = pd.read_csv(save_progress+"nouns_bynation.csv",index_col=0)
# verbs_bynation = utils.group_lemmacounts_bynation(verb_counts,list(stories.nation.unique())).to_csv(save_progress+"verbs_bynation.csv",index=True)
# verbs_bynation = pd.read_csv(save_progress+"verbs_bynation.csv",index_col=0)

In [195]:
nouns_bylangfam = nouns_bynation.T.copy().reset_index().rename(columns={'index':'Nation'})

In [199]:
nouns_bylangfam.loc[0,'Nation'] in grouped_regions[key]

False

In [204]:
nouns_bylangfam = nouns_bynation.T.copy().reset_index().rename(columns={'index':'Nation'})
lang_fam = []
for i in nouns_bylangfam.index:
    hasfam = False
    for key in grouped_regions.keys():
        if nouns_bylangfam.loc[i,'Nation'] in grouped_regions[key]:
            lang_fam.append(key)
            hasfam = True
    if not hasfam:
        lang_fam.append(np.nan)
nouns_bylangfam['language_family'] = lang_fam
nouns_bylangfam = nouns_bylangfam.groupby('language_family').mean().T
nouns_bylangfam.to_csv(save_progress+"nouns_bylangfam.csv",index=True)

In [203]:
nouns_bylangfam.T

language_family,albanian,armenic,austronesian,bantu,celtic,chadic,east_asia,germanic,hellenic,hindustani,italic,native_american,native_cana,semitic,slavic,turkic,uralic,urgic
lemma,,,,,,,,,,,,,,,,,,
king,0.0,0.0,0.000740,0.0,0.004363,0.0,0.002551,0.003541,0.009419,0.008664,0.004108,0.001221,0.0,0.0,0.003378,0.0,0.0,0.0
would,0.0,0.0,0.004792,0.0,0.004861,0.0,0.005725,0.004836,0.007731,0.005062,0.002462,0.007762,0.0,0.0,0.002480,0.0,0.0,0.0
princess,0.0,0.0,0.000296,0.0,0.001545,0.0,0.001751,0.001408,0.002133,0.002406,0.002019,0.000000,0.0,0.0,0.002368,0.0,0.0,0.0
could,0.0,0.0,0.003274,0.0,0.003762,0.0,0.004391,0.004301,0.006798,0.003195,0.001771,0.005866,0.0,0.0,0.001914,0.0,0.0,0.0
mother,0.0,0.0,0.001989,0.0,0.001304,0.0,0.002587,0.001709,0.003310,0.001660,0.000754,0.002365,0.0,0.0,0.001464,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
narran,0.0,0.0,0.000140,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
wyah,0.0,0.0,0.000140,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
mahthi,0.0,0.0,0.000140,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [ ]:
nouns_bynation = pd.read_csv(save_progress+"nouns_bynation.csv",index_col=0)
# verbs_bynation = pd.read_csv(save_progress+"verbs_bynation.csv",index_col=0)
nouns_bylangfam = pd.read_csv(save_progress+"nouns_bylangfam.csv",index_col=0)

## % of stories that mention x lemma

In [244]:
stories_lemmacount = pd.DataFrame()
for i in stories.index:
    lemma_count = pd.DataFrame(pd.Series(stories.iloc[i]['phrased'].strip("[]").replace('\'',"").split(", ")).value_counts())
    lemma_count.columns = ['sid'+str(i)]
    stories_lemmacount = pd.merge(stories_lemmacount,lemma_count,left_index=True,right_index=True,how="outer")
stories_lemmacount = stories_lemmacount.fillna(0)

stories_lemmacount = stories_lemmacount.reset_index()
stories_lemmacount = stories_lemmacount.rename(columns={'index':'lemma'})
stories_lemmacount = stories_lemmacount.set_index('lemma')
stories_lemmacount.to_csv(save_progress+"stories_lemmacount.csv")

In [14]:
stories_lemmacount = pd.read_csv(save_progress+"stories_lemmacount.csv",index_col=0)

In [591]:
stories_lemmacountT = stories_lemmacount.copy().T.reset_index()
stories_lemmacountT = stories_lemmacountT.rename(columns={'level_0':'sid'})
stories_lemmacountT = stories_lemmacountT.set_index('sid')
stories_lemmacountT = pd.merge(stories_lemmacountT,stories[['sid','nation','language_family']],left_index=True,right_on='sid')
stories_lemmacountT = stories_lemmacountT.rename(columns = {'nation_y':'nation'})

In [152]:
stories_lemmacount.columns[np.where(stories_lemmacount.loc['momotaro']!=0)]

Index(['sid0', 'sid2184', 'sid2577'], dtype='object')

In [290]:
nation_storycount = dict(stories.groupby('nation').count()['title'])

In [229]:
bigrams = pd.read_csv(data+'bigrams.csv').set_index('bigram').rename(columns={' count':'count'})
bigrams['count'] = bigrams['count'] + 0.00001

## percentile of group in lemma usage

count(lemma) / len(story words) --> avg freq --> percentile of usage

"nation a is in the top n% of users of lemma x"

In [387]:
stories_lemmafreq = stories_lemmacount.copy()
stories_lemmafreq = stories_lemmafreq.div(stories_lemmacount.sum(axis=1),axis=0)

In [582]:
stories_lemmafreq[stories_lemmafreq.columns[np.where(stories_lemmafreq.loc['momotaro']!=0)[0]]]

,sid0,sid2184,sid2577
lemma,,,
aalborg,0.0,0.0,0.0
aard,0.0,0.0,0.0
aarhus,0.0,0.0,0.0
aaron,0.0,0.0,0.0
aasvo,0.0,0.0,0.0
...,...,...,...
zuur,0.0,0.0,0.0
zuya,0.0,0.0,0.0
zuyder,0.0,0.0,0.0


In [595]:
# *! groupby
groupby = 'nation' # nation, language_family
wrt = 'nation' # nation/language_family, lemma

In [596]:
nation_lemmamean = pd.merge(stories_lemmafreq.T,stories[['sid','nation','language_family']],left_index=True,right_on='sid')
nation_lemmamean = nation_lemmamean.rename(columns={'nation_y':'nation'})
nation_lemmamean = nation_lemmamean.groupby(groupby).mean()
nation_lemmameanT= nation_lemmamean.T

nation_lemmamean_save = nation_lemmameanT
nation_lemmamean_save['englishness'] = lemma_englishness
nation_lemmamean_save['common'] = nation_lemmamentions[stories.nation.unique()].mean(axis=1)
nation_lemmamean_save.to_csv(save_progress+'nation_lemmamean.csv',index=True)

In [402]:
lemma_percentile = pd.DataFrame()
if wrt == 'nation' or wrt == 'language_family':
    for col in nation_lemmameanT:
        lemma_percentile[col] = [percentileofscore(nation_lemmameanT[col], value) for value in nation_lemmameanT[col]]
elif wrt == 'lemma':
    for col in nation_lemmamean:
        lemma_percentile[col] = [percentileofscore(nation_lemmamean[col], value) for value in nation_lemmamean[col]]
    lemma_percentile = lemma_percentile.T
lemma_percentile.columns = list(nation_lemmamean.index.values)
# lemma_percentile['englishness'] = lemma_englishness
# lemma_percentile['common'] = nation_lemmamentions[stories.nation.unique()].mean(axis=1)
lemma_percentile = lemma_percentile.rename(index=dict(zip(range(len(list(nation_lemmamean.columns.values))),list(nation_lemmamean.columns.values))))

In [562]:
nation_lemma_lemmapercentile['englishness'] = lemma_englishness
nation_lemma_lemmapercentile['common'] = nation_lemmamentions[stories.nation.unique()].mean(axis=1)

In [563]:
nation_lemma_lemmapercentile.to_csv(save_progress+"nation_lemma_lemmapercentile.csv",index=True)

In [403]:
lemma_percentile.to_csv(save_progress+f"{groupby}_lemma_{wrt}percentile.csv",index=True)

In [9]:
nation_lemma_lemmapercentile = pd.read_csv(save_progress+"nation_lemma_lemmapercentile.csv",index_col=0)
nation_lemma_nationpercentile = pd.read_csv(save_progress+"nation_lemma_nationpercentile.csv",index_col=0)

uniqueness to group = langfam/nation percentile <br>
uniqueness within group = lemma percentile <br>
<br>
lemma_uniquness = unique1 x unique2

In [551]:
lemma_unique_nation = pd.DataFrame(index=dict(zip(range(len(list(nation_lemmamean.columns.values))),list(nation_lemmamean.columns.values))))
for col in nation_lemma_lemmapercentile.columns:
    lemma_unique_nation[col] = nation_lemma_lemmapercentile[col].values+.5*nation_lemma_nationpercentile[col].values
lemma_unique_nation = lemma_unique_nation.rename(index=dict(zip(range(len(list(nation_lemmamean.columns.values))),list(nation_lemmamean.columns.values))))

In [552]:
lemma_unique_nation['englishness'] = lemma_englishness
lemma_unique_nation['common'] = nation_lemmamentions[stories.nation.unique()].mean(axis=1)

In [553]:
lemma_unique_nation.to_csv(save_progress+'lemma_unique_nation.csv',index=True)

### language family

In [359]:
langfam_lemmapercentile = pd.merge(stories_lemmafreq.T,stories[['sid','nation','language_family']],left_index=True,right_on='sid')
langfam_lemmapercentile = langfam_lemmapercentile.rename(columns={'nation_y':'nation'})
langfam_lemmapercentile = langfam_lemmapercentile.groupby('language_family').mean()
for col in langfam_lemmapercentile:
    langfam_lemmapercentile[col] = [percentileofscore(langfam_lemmapercentile[col], value) for value in langfam_lemmapercentile[col]]
langfam_lemmapercentile = langfam_lemmapercentile.T

In [376]:
langfam_lemmapercentile['englishness'] = lemma_englishness
langfam_lemmapercentile['common'] = langfam_lemmapercentile[stories.language_family.unique()].mean(axis=1)

In [377]:
langfam_lemmapercentile.to_csv(save_progress+"langfam_lemmapercentile.csv",index=True)

In [361]:
langfam_lemmapercentile = pd.read_csv(save_progress+"langfam_lemmapercentile.csv",index_col=0)
langfam_lemmapercentile.head(2)

,albanian,armenic,austronesian,bantu,celtic,chadic,east_asia,germanic,hellenic,hindustani,italic,native_american,native_cana,semitic,slavic,turkic,uralic,urgic
aalborg,50.000000,50.000000,50.000000,50.0,50.000000,50.000000,100.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
aard,44.444444,44.444444,94.444444,100.0,44.444444,44.444444,88.888889,44.444444,44.444444,44.444444,44.444444,44.444444,44.444444,44.444444,44.444444,44.444444,44.444444,44.444444


# LIWC analysis

In [7]:
liwc_dict = utils.readDict(data+'LIWC2007_English100131.dic')
# power = ['king','queen','prince','princess','servant','master','lord','throne','emperor','palace','sultan','royal','majesty','tsar']
royalty = ['king*','queen*','prince*','princess*','servant*','master*','lord*','throne*','emperor*','palace*','sultan*','royal*','majesty*','tsar*']
hero = ['hero*','warrior*','knight*','beast*','brave*','courage*']
edited_dict = liwc_dict[0].copy()
# for word in power:
#     if word in edited_dict.keys():
#         edited_dict[word] += ['power']
#     else:
#         edited_dict[word] = ['power']
for word in royalty:
    if word in edited_dict.keys():
        edited_dict[word] += ['royalty']
    else:
        edited_dict[word] = ['royalty']
for word in hero:
    if word in edited_dict.keys():
        edited_dict[word] += ['hero']
    else:
        edited_dict[word] = ['hero']

edited_values = [['funct'], ['pronoun'], ['ppron'], ['i'], ['we'], ['you'], ['shehe'], ['they'], ['ipron'], ['article'], ['verb'], ['auxverb'], ['past'], ['present'], ['future'], ['adverb'], ['preps'], ['conj'], ['negate'], ['quant'], ['number'], ['swear'], ['social'], ['family'], ['friend'], ['humans'], ['affect'], ['posemo'], ['negemo'], ['anx'], ['anger'], ['sad'], ['cogmech'], ['insight'], ['cause'], ['discrep'], ['tentat'], ['certain'], ['inhib'], ['incl'], ['excl'], ['percept'], ['see'], ['hear'], ['feel'], ['bio'], ['body'], ['health'], ['sexual'], ['ingest'], ['relativ'], ['motion'], ['space'], ['time'], ['work'], ['achieve'], ['leisure'], ['home'], ['money'], ['relig'], ['death'], ['assent'], ['nonfl'], ['filler'],
['power'], ['royalty'], ['hero']]
helper_odict = collections.OrderedDict()
for i in range(len(edited_values)):
    helper_odict[i] = edited_values[i]

mydict = (edited_dict, helper_odict.values())
# animals: horse, bird, dragon, beast, witch, fish, tortoise, lion, puma, snake, rabbit, mouse, hare, stag, serpent

In [6]:
liwc_dict

(OrderedDict([('a', ['funct', 'article']),
              ('abandon*', ['affect', 'negemo', 'sad', 'cogmech', 'inhib']),
              ('abdomen*', ['bio', 'body']),
              ('abilit*', ['achieve']),
              ('able*', ['achieve']),
              ('abortion*', ['bio', 'health', 'sexual']),
              ('about', ['funct', 'adverb', 'preps']),
              ('above', ['funct', 'preps', 'space', 'relativ']),
              ('abrupt*', ['time', 'relativ']),
              ('abs', ['bio', 'body']),
              ('absent*', ['work']),
              ('absolute', ['cogmech', 'certain']),
              ('absolutely',
               ['funct', 'adverb', 'cogmech', 'certain', 'assent']),
              ('abstain*', ['cogmech', 'inhib']),
              ('abuse*', ['affect', 'negemo', 'anger']),
              ('abusi*', ['affect', 'negemo', 'anger']),
              ('academ*', ['work']),
              ('accept', ['affect', 'posemo', 'cogmech', 'insight']),
              ('accepta*', ['affe

In [148]:
stories_liwc = pd.DataFrame()
for i in stories.index:
    text = stories['text'][i]
    liwc_counts = utils.wordCount(text, mydict)
    liwc_df = pd.DataFrame(list(liwc_counts[0].items())).set_index(0) / liwc_counts[2]
    liwc_df.columns = [stories.loc[i,'sid']]
    stories_liwc = pd.concat([stories_liwc,liwc_df],axis=1)
stories_liwc = stories_liwc.T

In [149]:
nation_liwc = pd.merge(stories_hofstede,stories_liwc,right_index=True,left_on='sid').set_index('sid')
nation_liwc = nation_liwc.groupby('nation').mean().reset_index()
nation_liwc.columns
hofstede_cols = ['pdi','idv','mas','uai','ltowvs','ivr']
liwc_cols = nation_liwc.columns[8:]
for col in liwc_cols:
    nation_liwc[col] = (nation_liwc[col] - nation_liwc[col].mean())/nation_liwc[col].std()

nation_liwc.to_csv(save_progress+"nation_liwc.csv",index=True)

# PCA

In [228]:
# doc_corpus = [models.doc2vec.TaggedDocument(words=l, tags=[str(i)]) for i, l in enumerate(lemmas)]
# doc_model = models.doc2vec.Doc2Vec(doc_corpus, min_count=200, window=10, workers=8)
# doc_model.save(save_progress+'doc2vec.model')
# doc_model = models.doc2vec.Doc2Vec.load(save_progress+'doc2vec.model')
# tsne = utils.doc2vec_tsne(doc_model, stories['language_family'])
tsne = pd.merge(tsne, stories[['nation','language_family']], left_index=True, right_index=True, how='left')
tsne = tsne.rename(columns={'nation':'Nation'})
tsne['Nation'] = tsne['Nation'].str.title()
tsne['language_family'] = tsne['language_family'].str.title()

In [229]:
tsne.to_csv(save_progress+"tsne.csv",index=True)

In [230]:
tsne

,X1,X2,Nation,language_family
0,-17.040495,6.338946,Japanese,East_Asia
1,-1.613154,23.311752,Serbian,Slavic
2,-7.550614,-11.772786,German,Germanic
3,28.067266,30.262329,French,Italic
4,-18.574621,33.876457,North_American_Native,Native_American
...,...,...,...,...
2927,-13.959932,22.061859,Australian_Ethnic,Austronesian
2928,4.039639,13.369742,Scottish,Celtic
2929,-17.184561,-11.282936,Italian,Italic
2930,5.765532,-33.906895,Norwegian,Germanic


In [ ]:
pca20 = utils.doc2vec_tsne(doc_model, stories['language_family'], 20)
pca20 = pd.merge(pca20, stories[['nation','language_family']], left_index=True, right_index=True, how='left')
pca20 = pca20.rename(columns={'nation':'Nation'})
pca20['Nation'] = pca20['Nation'].str.title()


In [219]:
new_vals = utils.doc2vec_tsne(doc_model, stories['language_family'], 20)

In [220]:
new_vals

array([[-17.040495 ,   6.3389463],
       [ -1.613154 ,  23.311752 ],
       [ -7.5506144, -11.772786 ],
       ...,
       [-17.18456  , -11.282936 ],
       [  5.7655325, -33.906895 ],
       [  5.4170303,   4.7176046]], dtype=float32)

# Topic modelling

In [ ]:
dictionary = corpora.Dictionary(lemmas.sample(frac=0.5))
bow_corpus = [dictionary.doc2bow(text) for text in lemmas]